In [ ]:
#Copy dataset from Google drive to session storage

zip_path_train = '/content/drive/My Drive/Python/SUBMISSION FOLDER/Final_test_data.zip'
!cp '{zip_path_train}' .
!unzip -q Final_test_data.zip
!rm Final_test_data.zip

In [ ]:
#to copy saved model of Caption Generator from drive to session storage

path='/content/drive/My Drive/Python/Models_caption/model_rerun9.h5'
!cp '{path}' .

In [ ]:
#Copy Features extracted pickle file using Xception model from Google drive to session storage

path='/content/drive/My Drive/Python/SUBMISSION FOLDER/features.p'
!cp '{path}' .

In [ ]:
#Copy Tokenizer pickle file from Google drive to session storage

path='/content/drive/My Drive/Python/SUBMISSION FOLDER/tokenizer.p'
!cp '{path}' .

In [ ]:
#Copy NSFW model from Google drive to session storage

model_source= '/content/drive/My Drive/Python/SUBMISSION FOLDER/weights-improvement-VGG16 unlkd-09-0.89.hdf5'
!cp '{model_source}' .

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import argparse

import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

#Initial preprocessing to meet the needs of Image Captioning Model

def preprocess_extract(filename, model):
        try:
            photo = Image.open(filename)    #Read Images
        except:
            print("Trouble opening image file, please ensure the path provided is correct")
        photo = photo.resize((299,299))
        photo = np.array(photo)
        if photo.shape[2] == 4: 
            photo = photo[..., :3]
        photo = np.expand_dims(photo, axis=0)
        photo = photo/127.5
        photo = photo - 1.0
        feature = model.predict(photo)    #Extract features using model
        return feature


def tokenize_words(integer, tokenizer):   #Tokenize words using pickle file
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

  
def caption_creator(model, tokenizer, photo, max_length):   #Final Captions are Created Here
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = tokenize_words(pred, tokenizer)    
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [ ]:
#Filling Variables used For Captioning Model

def generate_caption(filename):
  path = filename   #Image Path
  max_length = 32   #Max Length of Descriptions 
  tokenizer = load(open("tokenizer.p","rb"))  #Insert Path for Tokenizer Pickle File
  model = load_model('model_rerun9.h5')       #Insert Path for Image Captioning Model
  model_xcept = Xception(include_top=False, pooling="avg")  #Xception Model
  photo = preprocess_extract(path, model_xcept)  
  image = Image.open(path)                          
  caption = caption_creator(model, tokenizer, photo, max_length)  #Final Caption Creation
  print("\n\n")
  print(caption)

In [ ]:
#Setting the Location of the Database

test_root='/content/Final_test_data'   #Insert Final Dataset Path Here

In [ ]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
from keras.preprocessing import image

In [ ]:
#Shuffling the Images and Making a NumPy Array of All the Images(NSFW + SAFE)

test_files = np.array(os.listdir(test_root))

for _ in range(5):
    np.random.shuffle(test_files)

test_image_files = []
for image in test_files:
    test_image_files.append(os.path.join(test_root, image))

test_image_files = np.array(test_image_files)
for _ in range(5): indexes = np.random.permutation(range(len(test_image_files)))
test_image_files = test_image_files[indexes]
test_image_files[:10]

array(['/content/Final_test_data/3463034205_e541313038.jpg',
       '/content/Final_test_data/1412832223_99e8b4701a.jpg',
       '/content/Final_test_data/3422394336_e465f60b7c.jpg',
       '/content/Final_test_data/prefix_GantMan_0000B748-551C-4388-A4EC-B1C832324C6F.jpg.jpeg',
       '/content/Final_test_data/prefix_GantMan_00000A77-D61D-47FC-B4A6-934E1E5ED671.jpg.jpeg',
       '/content/Final_test_data/10815824_2997e03d76.jpg',
       '/content/Final_test_data/224026428_0165164ceb.jpg',
       '/content/Final_test_data/697582336_601462e052.jpg',
       '/content/Final_test_data/2672354635_3a03f76486.jpg',
       '/content/Final_test_data/3183875944_b2be694e06.jpg'], dtype='<U85')

In [ ]:
#Declaring Variables Required By NSFW Filter

IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS, BATCH_SIZE = 150, 150, 3, 32

In [ ]:
from tensorflow import keras

In [ ]:
#Loading NSFW Model

model_xfer2=keras.models.load_model('/content/weights-improvement-VGG16 unlkd-09-0.89.hdf5')  #Insert NSFW Model Path Here

In [ ]:
#Preprocessing Images According to NSFW Model Needs And Prediction with Caption Generation
NSFW_img=0
from keras.preprocessing import image
for test_image in tqdm(test_image_files):

      img = image.load_img(test_image, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x /= 255.0

      images_list = np.vstack([x])
      classes = model_xfer2.predict(images_list, batch_size=10)
      prob = classes[0]
      actual_name = (test_image.split(os.path.sep)[-2].split('.')[0]).upper() 
      pred_name = 'SAFE' if (prob >= 0.5) else 'NSFW'
      is_correct = (pred_name =='SAFE')
      if is_correct:
        #print(test_image)
        generate_caption(test_image)
        plt.imshow(img)
        plt.show(img)
        print('The above image is '+pred_name+' and does not contain any NSFW element')
        print('NSFW Images Omitted='+str(NSFW_img))
      else:
        NSFW_img=NSFW_img+1

Output hidden; open in https://colab.research.google.com to view.